In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import re
import string
import shutil
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, GlobalMaxPooling1D, Conv1D, Dropout, TextVectorization
from tensorflow.keras.losses import BinaryCrossentropy
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [ ]:
train = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")
sample_submission = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
sample_submission.head()

In [ ]:
print(f"train shape {train.shape}")
print(f"test shape {test.shape}")

In [ ]:
train.describe()

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
train.isnull().sum()

In [ ]:
train.keyword.value_counts().to_frame()

In [ ]:
train.location.value_counts().to_frame()

In [ ]:
train.target.value_counts().to_frame()

In [ ]:
def custom_tokenize(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    stripped_hyperlink = tf.strings.regex_replace(stripped_html, 'https?://\S+|www\.\S+', '')
    stripped_square_brackets = tf.strings.regex_replace(stripped_hyperlink, '\[.*?\]', '')
    stripped_punctuation = tf.strings.regex_replace(stripped_square_brackets, '[%s]' % re.escape(string.punctuation), '')
    stripped_punctuation = tf.strings.regex_replace(stripped_punctuation, '\n', '')
    stripped_number = tf.strings.regex_replace(stripped_punctuation, '\w*\d\w*', '')
    
    return tf.strings.regex_replace(stripped_number, '[%s]' % re.escape(string.punctuation), '')

In [ ]:
train['text'] = custom_tokenize(train['text'])

In [ ]:
#hyperParam
max_features = 1000
sequence_length = 75

In [ ]:
vectorize_layer = TextVectorization(
                    standardize = custom_tokenize,
                    max_tokens = max_features,
                    output_mode = 'int',
                    output_sequence_length=sequence_length)

In [ ]:
train['text'].values

In [ ]:
vectorize_layer.adapt(train['text'].values)

In [ ]:
vectorize_text = vectorize_layer(train['text'].values)
vectorize_text

In [ ]:
Xs = vectorize_text
y = train['target']

In [ ]:
#hyperparam
embedding_dim = 32
filters = 128
kernel_size = 5
dense_dim = 256

In [ ]:
model = Sequential([
    Embedding(max_features + 1, embedding_dim),
    Conv1D(filters=filters, kernel_size=kernel_size, activation='relu'),
    GlobalMaxPooling1D(),
    Dropout(0.2),
    Dense(dense_dim, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='Adamax', metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

In [ ]:
model.summary()

In [ ]:
NUM_EPOCHS = 15
VAL_SPLIT = 0.2

In [ ]:
history = model.fit(Xs, y, epochs=NUM_EPOCHS, validation_split=VAL_SPLIT)

In [ ]:
history_dict = history.history
history_dict.keys()

In [ ]:
plt.plot(history.history['accuracy'], color='b')
plt.plot(history.history['val_accuracy'], color='r')

plt.title('Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend('Training', 'Validation')

plt.show()

In [ ]:
plt.plot(history.history['loss'], color='g')
plt.plot(history.history['val_loss'], color='r')

plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('loss')
plt.legend('Training', 'Validation')

plt.show()

In [ ]:
test['text'] = custom_tokenize(test['text'])

In [ ]:
import random

random_num = random.randint(0, (3263-1))

data = test['text'][random_num]

input_data = vectorize_layer(data)

predictions = model.predict(np.expand_dims(input_data, 0))

print(data)
print(predictions)

In [ ]:
value = predictions[0][0]
value > 0.5

In [ ]:
if value>0.5 or value == 0.5:
    print('Disaster Tweets')
elif value < 0.5:
    print('Not a Disaster Tweets')

In [ ]:
submission = sample_submission
submission

In [ ]:
submit_data = test['text']
input_submit = vectorize_layer(submit_data)
prediksi = model.predict(input_submit)

In [ ]:
prediksi_target = []
for pred in prediksi:
    if pred >= 0.5:
        prediksi_target.append(1)
    else:
        prediksi_target.append(0)

In [ ]:
submission = pd.DataFrame({'id':test['id'].values.tolist(), 'target':prediksi_target})

In [ ]:
submission.to_csv('submission.csv', index=False)